## Dynamic Generation of Transducible Functions

In [ ]:
from pydantic import BaseModel
from typing import Optional
from agentics.core.atype import *
from agentics.core.transducible_functions import With

class GenericInput(BaseModel):
    content: Optional[str] = None


class Email(BaseModel):
    "bla bla"
    to: Optional[str] = Field(None,description="bla bla")
    subject:Optional[str]=None
    body: Optional[str]=None

### Dynamic Generation

Transducible functions can be dynamically generared in two ways:
1) Using make_transducible_from_types method
2) Using << operator between any two pydantic types

## Composition


In [ ]:
class Summary(BaseModel):
    summary_text:Optional[str]
 
input = GenericInput(content=
    "Write news story on Zoran Mandani won the Election"
    "in NYC and send it to Alfio")

write_mail = Email<<GenericInput
mail = await write_mail(input)
print(mail.model_dump_json(indent=2))

summary_composite_1 = Summary << write_mail
mail = await summary_composite_1(input)
print(mail.model_dump_json(indent=2))

summary_composite_2 = Summary <<(Email<<GenericInput)
mail = await summary_composite_2(input)
print(mail.model_dump_json(indent=2))

In [ ]:
summarize = Summary << With(
    Email,
    instructions="Rewrite the email into a summary",
    enforce_output_type=True,
    verbose_transduction=False,
)
write_mail =(Email<<GenericInput)
mail = await write_mail(GenericInput(content="Zoran Mandani won the Election in NYC"))
print(await summarize(mail))

In [ ]:

from agentics import AG
from pydantic import BaseModel, Field
from typing import Optional
import agentics.core.transducible_functions


class Movie(BaseModel):
    movie_name: Optional[str] = None
    description: Optional[str] = None
    year: Optional[int] = None

class Genre(BaseModel):
    genre:Optional[str] = Field(
        None,
        description="return one category,   "
                    "e.g. comedy, drama, ...")


movies = AG.from_csv(
    "data/movies.csv", atype=Movie, max_rows=2
) 
movie = movies[1]

In [ ]:
classify_genre= Genre << Movie
genre = await classify_genre(movie)
print((genre @ movie).model_dump_json(indent=2))

In [ ]:
await (Genre << movie)

In [ ]:
classify_genre= Genre << With(Movie,
                            provide_explanation=True)
genre, explanation = await classify_genre(movie)
print(genre.model_dump_json(indent=2))
print(explanation.model_dump_json(indent=2))